In [108]:
import numpy as np
import random
class Affine:
    def __init__(self,W,B):
        self.params=[W,B]
        self.grads=[np.zeros_like(W),np.zeros_like(B)]
        self.x=None
    def forward(self,x):
        self.x=x
        W,B=self.params
        out=np.dot(x,W)+B
        if out>0:
            return 1
        else:
            return 0
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

class SGD:
    def __init__(self,lr=0.01):
        self.lr = lr
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]
class StepWithLoss:
    def __init__(self):
        self.params,self.grads=[],[]
        self.loss=None
        self.y=None
        self.t=None
    def forward(self, x, t):
        self.t = t
        if x>0:
            self.y=1
        else:
            self.y=0

    def backward(self, dout=1):
        dx = (self.y - self.t) * dout
        return dx

In [78]:

class SimpleNet:
    def __init__(self):
        W1 = 0.01 * np.random.randn(2, 1)
        b1 = np.zeros(1)

        # レイヤの生成
        self.layers = [
            Affine(W1, b1),
        ]
        self.loss_layer = StepWithLoss()
        # すべての重みと勾配をリストにまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        if x>0:
            return 1
        else:
            return 0

    def forward(self, x, t):
        score = self.predict(x)
        self.loss_layer.forward(score, t)

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

In [134]:
model=SimpleNet()
opt=SGD()

In [135]:
data=[[np.array([[0,0]]),np.array([[0]])],
      [np.array([[0,1]]),np.array([[1]])],
      [np.array([[1,0]]),np.array([[1]])],
      [np.array([[1,1]]),np.array([[0]])],
     ]
for i in range(1000):
    randomNum=random.randint(0,3)
    model.forward(data[randomNum][0],data[randomNum][1])
    model.backward()
    opt.update(model.params,model.grads)

In [139]:
model.predict(np.array([1,1]))

0